In [1]:
import cv2 as cv
import pytesseract

In [2]:
pytesseract.pytesseract.tesseract_cmd = '/usr/local/Cellar/tesseract/5.1.0/bin/tesseract'

In [137]:
def matToText(filename, write=False):
  mat = cv.imread(filename)

  gray = cv.cvtColor(mat, cv.COLOR_BGR2GRAY)
  blured = cv.GaussianBlur(gray, (15, 3), 0, 0)
  th, threshed = cv.threshold(blured, 10, 255, cv.THRESH_BINARY_INV|cv.THRESH_OTSU)
  kernel = cv.getStructuringElement(cv.MORPH_RECT, (10, 3))
  threshed = cv.dilate(threshed, kernel)
  hist = cv.reduce(threshed,1, cv.REDUCE_AVG).reshape(-1)

  h,w = mat.shape[:2]
  stack_y = []
  for ii in range(h-1):
    if (hist[ii] and not hist[ii+1]):
      stack_y.append(ii)
    elif (not hist[ii] and hist[ii+1]):
      stack_y.append(ii)

  ocr_text = []
  while(stack_y):
    y1 = stack_y.pop()
    y2 = stack_y.pop()
    nonZero = cv.findNonZero(threshed[y2:y1, :])
    xb,_,wb,_ = cv.boundingRect(nonZero)
    croped = mat[y2:y1, xb:xb+wb]
    cv.rectangle(mat, (xb, y1), (xb+wb, y2), (255, 155, 0))
    ocr_text.append(pytesseract.image_to_string(croped, lang='tha'))
  filename = filename[filename.rfind('/')+1:filename.rfind('.')]
  if (write):
    print(filename)
    cv.imwrite(f"drawed-{filename}.jpg", mat)
    cv.imwrite(f"thresed-{filename}.jpg", threshed)

  return ocr_text[::-1]

In [138]:
pages = []
for i in range(27, 30):
  filename = "./bud-images/65075-{:03d}.jpg".format(i)
  pages.append(matToText(filename, True))

65075-027
65075-028
65075-029
